# Find, Load, and Visualise Earth Observation Imagery

This notebook demonstrates how to find, load, and visualise Earth observation imagery using cloud native approaches, which work well on your desktop or in cloud environments!

The notebook demonstrates how to find images from the Digital Earth Australia STAC catalog, load them using `odc-stac`, and visualise them using extensions to xarray provided by `odc-geo`.

## Set up

The first step is to set up the required Python libraries and local imports.

* `odc.stac` and `pystac_client` are used to access Digital Earth Australia's STAC catalog
* `numpy` is used to manipulate data
* `odc.geo.xr` enables additional functionality for xarrays

In [ ]:
from odc.stac import configure_s3_access, load
from pystac_client import Client as PystacClient
import numpy as np
import odc.geo.xr  # noqa: F401

The second step is to start a Dask client.

Dask supports local parallel processing and can help speed up computation times.

In [ ]:
from dask.distributed import Client as DaskClient

dask_client = DaskClient()
dask_client

## Part 1: Find

### 1.1 Connecting to the catalog

In [ ]:
# The catalog URL for DEA's STAC catalog
catalog = "https://explorer.dea.ga.gov.au/stac"

# Pystac_client's Client class is used to connect to the catalog
stac_client = PystacClient.open(catalog)

# Configure settings for reading from DEA's STAC
configure_s3_access(
    cloud_defaults=True,
    aws_unsigned=True,
)

### 1.2 Selecting an area to query

In [ ]:
from eo_insights.ui import select_on_a_map
from ipyleaflet import basemaps, basemap_to_tiles

basemap = basemap_to_tiles(basemaps.Esri.WorldImagery)
geom = select_on_a_map(height="500px", layers=(basemap,), center=(-26, 135), zoom=4)

### 1.3 Set year and month to query

In [ ]:
# Set a start and end date
date_query = "2021-12"

### 1.4 Choose collections and filters

In [ ]:
# Set DEA product ID as the STAC "Collection"
collections_query = ["ga_s2am_ard_3", "ga_s2bm_ard_3"]

# Set up a filter query. The s2cloudless:cloud variable is a DEA-specific metadata field
filter_query = "s2cloudless:cloud < 20"

### 1.5 Running the query to indentify matching STAC items

In [ ]:
# Query with filtering for cloud cover
items = stac_client.search(
    intersects=geom,
    collections=collections_query,
    datetime=date_query,
    filter=filter_query,
).item_collection()

print(f"Found {len(items)} items")

## Load

### 2.1 Using odc-stac to load identified items

This may take a few minutes

In [ ]:
# Load our filtered data
ds_filtered = load(
    items,
    bands=["nbart_red", "nbart_green", "nbart_blue"],
    crs="utm",
    chunks={},
    resolution=30,
    groupby="solar_day",
    geopolygon=geom,
).compute()

ds_filtered

### 2.2 Review loaded imagery

Identify which image you want to export and note the date.

In [ ]:
# To_array sets up a 3D array with the time dimension, which works directly
# with the plot function to make an RGB image
ds_filtered.to_array().plot.imshow(col="time", col_wrap=3)

## Part 3: Visualise

### 3.1 Select best image

Update the `best_image_date` parameter to match the date you identified in the previous step.

In [ ]:
best_image_date = "2021-12-20"

best_image = ds_filtered.sel(time=best_image_date).squeeze()

best_image

### 3.2 View the selected image on an interactive map

In [ ]:
visualisation = best_image.odc.to_rgba()

visualisation.odc.explore()

### 3.3 Improve the image contrast

Calculate the values corresponding to the 1st and 99th percentiles. 
These can be used in the `to_rgba()` function to stretch the image.

In [ ]:
percentile_stretch = (1, 99)

rgb_array = best_image.to_array().values

stretch_vmin, stretch_vmax = np.nanpercentile(rgb_array, percentile_stretch)

Apply the percentile stretch values to the visualisation

In [ ]:
stretch_visualisation = best_image.odc.to_rgba(vmin=stretch_vmin, vmax=stretch_vmax)

stretch_visualisation.odc.explore()

### 3.4 Export to a cloud-optimised GeoTIFF

In [ ]:
stretch_visualisation.odc.write_cog("sentinel2_example.tif", overwrite=True)